In [28]:
# Import statements
import glob
from spectral_cube import SpectralCube
import astropy.units as u
import os

In [24]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Use dask for parallelization
dask = True

if dask:
    from dask.diagnostics import ProgressBar
    pbar = ProgressBar()
    pbar.register()

In [3]:
# Get a cube and the pb file for that cube
freq_spw = '273_spw51'
pb_fn = glob.glob(f"/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/source_ab*{freq_spw}*clean*.pb")
cube_fn = glob.glob(f"/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/source_ab*{freq_spw}*clean*.image") # We need the units of this... and no one knows why
pb = SpectralCube.read(pb_fn[0], format = 'casa_image')
cube = SpectralCube.read(cube_fn[0], format = 'casa_image')

/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]
/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


In [4]:
cube

DaskVaryingResolutionSpectralCube with shape=(3840, 512, 512) and unit=Jy / beam and chunk size (128, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   3840  type_s: FREQ      unit_s: Hz     range: 273010861630.481 Hz:274885343259.334 Hz

In [5]:
pb

DaskSpectralCube with shape=(3840, 512, 512) and chunk size (128, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   3840  type_s: FREQ      unit_s: Hz     range: 273010861630.481 Hz:274885343259.334 Hz

In [16]:
# Use division to correct the cube, and deal with the units issue
cube_pbcor = cube.unitless/(pb.unitless)
cube_pbcor_unit = cube_pbcor * (u.Jy/u.beam)

In [17]:
cube_pbcor_unit

DaskVaryingResolutionSpectralCube with shape=(3840, 512, 512) and unit=Jy / beam and chunk size (128, 256, 512):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:   3840  type_s: FREQ      unit_s: Hz     range: 273010861630.481 Hz:274885343259.334 Hz

In [21]:
outfile = cube_fn[0].replace(".image",".pbcor.image")
outfile

'/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/source_ab_273_spw51_clean_2sigma_n50000_masked_3sigma_pbmask0p18.pbcor.image'

In [33]:
if not os.path.exists(outfile):
    cube_pbcor_unit.write(outfile, format='fits')  